# Multivariante del método K-Nearest Neighbors

Hasta ahora exploramos como usar un modelo de ML, el KNN simple que uso solo una caracteristica para las predicciones. Si bien esto ayudo a familiarizarse con los conceptos basicos del ML, esta claro que usar una sola caracteristica no refleja la realidad.<br>

Hay 2 formas en las que podemos retocar este modelo para tratar de mejorar la precision (mejorar el RMSE):<br>
- Incrementar el numero de atributos (**caracteristicas**) que el modelo usa para calcular similitudes cuando ranque los vecinos mas cercanos.
- Incrementar **k**, el numero de vecinos cercanos que el modelo usa.

### Eliminacion de caracteristicas no utiles y _**Normalizacion**_ de los datos.

Al seleccionar mas atributos, tenemos que tener cuidado con las columnas que no funcionan bien con la ecuacion de distancia:
- Valores no numericos (ejemplo, ciudad o estado)
    - La ecuacion de distancia euclidiana espera valores numericos.
- Valores perdidos
    - La ecuacion de la distancia euclideana espera un valor por cada observacion y atributo.
- Valores no ordinales (por ejemplo, latitud o longitud)
    - La clasifiacion por distancia euclideana no tiene sentido si todos los valores no son ordinales. 

In [22]:
import pandas as pd
import numpy as np

# Leyendo datos y acomodandolos aleatoreamente
dc_listings = pd.read_csv('dc_airbnb.csv')
dc_listings = dc_listings.loc[ np.random.permutation(len(dc_listings)) ]

# Convirtiendo la columna price a float (limpiando datos)
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
2021,100%,100%,1,4,Entire home/apt,1.0,1.0,2.0,130.0,$45.00,NaN,2,1125,90,38.893268,-76.990191,"Washington, D.C.",20002,DC
768,88%,88%,1,2,Private room,1.0,1.0,1.0,55.0,NaN,NaN,3,1125,4,38.906128,-76.996538,Washington,20002,DC
1140,100%,80%,4,2,Entire home/apt,1.0,1.0,1.0,215.0,$130.00,$500.00,2,180,65,38.911465,-77.036361,Washington,20005,DC
2760,NaN,NaN,1,2,Entire home/apt,2.0,1.0,2.0,500.0,NaN,NaN,1,1125,0,38.934429,-77.024223,Washington,20010,DC
1088,NaN,NaN,1,1,Shared room,1.0,1.0,1.0,100.0,NaN,NaN,2,1125,0,38.912634,-77.043345,Washington,20009,DC


In [23]:
# Mostrando informacion general de los datos. 
# Como: columnas, cuantos valores no nulos contiene cada columna y el tipo de dato
print(dc_listings.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3723 entries, 2021 to 2298
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    3289 non-null   object 
 1   host_acceptance_rate  3109 non-null   object 
 2   host_listings_count   3723 non-null   int64  
 3   accommodates          3723 non-null   int64  
 4   room_type             3723 non-null   object 
 5   bedrooms              3702 non-null   float64
 6   bathrooms             3696 non-null   float64
 7   beds                  3712 non-null   float64
 8   price                 3723 non-null   float64
 9   cleaning_fee          2335 non-null   object 
 10  security_deposit      1426 non-null   object 
 11  minimum_nights        3723 non-null   int64  
 12  maximum_nights        3723 non-null   int64  
 13  number_of_reviews     3723 non-null   int64  
 14  latitude              3723 non-null   float64
 15  longitude             3

Como podemos observar: 
- Hay columnas que contienen valores _**NO númericos**_:
    - _room_type_
    - _city_
    - _state_
- Y columnas con valores númericos pero _**NO ordinales**_:
    - _latitude_
    - _longitude_
    - _zipcode_


A su vez tambien evitaremos cualquier columna que no describa directamente el espacio vital:
- _host_response_rate_
- _host_acceptance_rate_
- _host_listings_count_


In [24]:
# Eliminando columnas no utiles.
drop_columns = [ 'room_type', 'city', 'state', 'latitude', 'longitude', 'zipcode', 'host_response_rate', 'host_acceptance_rate', 'host_listings_count' ]
dc_listings = dc_listings.drop(drop_columns, axis=1)
dc_listings.isnull().sum()

accommodates            0
bedrooms               21
bathrooms              27
beds                   11
price                   0
cleaning_fee         1388
security_deposit     2297
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64

De las columnas restantes, hay algunas que tienen pocos valores nulos:
- _bedrooms_
- _bathrooms_
- _beds_

De las cuales podemos eliminar esas filas sin perder mucha informacion.

Pero tambien hay 2 columnas con un gran numero de valores perdidos:
- _cleaning_fee_
- _security_deposit_

En este caso no podemos eliminar simplemente las filas, ya que perderiamos mucha informacion. Por ello, vamos a eliminar <br>
por completo esas dos columas.

In [25]:
# Eliminando columnas con un gran numero de valores perdidos
dc_listings = dc_listings.drop([ 'cleaning_fee', 'security_deposit' ], axis=1)

# Eliminando filas con valores nulos
dc_listings = dc_listings.dropna(axis=0)

dc_listings.isnull().sum()

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64

#### Normalizacion de Columnas

In [26]:
dc_listings.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
2021,4,1.0,1.0,2.0,130.0,2,1125,90
768,2,1.0,1.0,1.0,55.0,3,1125,4
1140,2,1.0,1.0,1.0,215.0,2,180,65
2760,2,2.0,1.0,2.0,500.0,1,1125,0
1088,1,1.0,1.0,1.0,100.0,2,1125,0


Como se puede observar, hay columnas con valores con mucha varianza, valores desmesurados. Y debido a como se calcula la _distancia euclideana_ <br>
podria ocasionar errores de distancia. 
Para evitar que una sola columna tenga demasiado impacto en la distancia, podemos _**normalizar**_ todas las columnas para que tengan una media de 0 <br>
y una desviacion estandar de 1.

_La **normalización** de los valores de cada columna preserva la distribucion de los valores de cada columana a su vez que alinea las escalas._

Esta es la formúla matemática que describe la transformacion que debe aplicarse a todos los valores de la columna: $$ Z_{i} = \frac{X_{i} - \mu}{\sigma} $$

Donde:
- $X_{i}$  es in valor de una columna especifica.
- $\mu$  es la media de todos los valores de la columna.
- $\sigma$  es a desviación estandar de todos los valores de la columna.

In [27]:
# Probando la normalizacion para entender la teoria.
first_transform = dc_listings['maximum_nights'] - dc_listings['maximum_nights'].mean()
normalized_column = first_transform / first_transform.std()
print(normalized_column.head(3))

# O de otra manera
normalized_column = first_transform / dc_listings['maximum_nights'].std()
print(normalized_column.head(3))

2021   -0.016573
768    -0.016573
1140   -0.016599
Name: maximum_nights, dtype: float64
2021   -0.016573
768    -0.016573
1140   -0.016599
Name: maximum_nights, dtype: float64


##### Aplicando esta trandormacion a todas las columnas de un DataFrame
Para ello se puede usar los metodos **mean()** y **std()** del DataFrame. Al utilizar estos métodos, se utilizan las medias y las desviaciones estándar de las columnas apropiadas para cada valor del DataFrame.

In [28]:
normalized_listings = ( dc_listings - dc_listings.mean() ) / (dc_listings.std())
normalized_listings['price'] =  dc_listings['price']
normalized_listings.head(3)

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
2021,0.401366,-0.249467,-0.439151,0.297345,130.0,-0.065038,-0.016573,2.561629
768,-0.596544,-0.249467,-0.439151,-0.546858,55.0,0.211298,-0.016573,-0.379894
1140,-0.596544,-0.249467,-0.439151,-0.546858,215.0,-0.065038,-0.016599,1.706535


### Calcúlo de la Distancia Euclideana para el _**Caso Multivariante**_

Anteriormente utilizamos dos modelos univariantes de KNN, utilizando el atributo _**accomodates**_ para el primero y _**bathrooms**_ para el segundo. <br>
Ahora entrenaremos un modelo con ambos atributos. Recordando que la ecuacion de la _distancia euclideana_ esta definida como: $$ d = \sqrt{(q_{1}-p_{1})^{2} + (q_{2}-p_{2})^{2} + ... + (q_{n}-p_{n})^{2}} $$
Y bien para hacer el calculo en Python, ahora nos apoyaremos del modulo _**scipy**_, utilizando la funcion _**distance.euclidean()**_:

In [29]:
from scipy.spatial import distance

first_listing = [ -0.596544, -0.439151 ]
second_listing = [ -0.596544, -0.412923 ]

dist = distance.euclidean( first_listing, second_listing )

La funcion euclidean(), espera:
- Dos vectores, los cuales se deben representar con un objeto tipo _lista_ (lista de Python, array de NumPy o serie de Pandas).
- Ambos deben ser unidimensionales y tener el mismo numero de elementos.

In [30]:
from scipy.spatial import distance
first_listing = normalized_listings.iloc[0][ ['accommodates', 'bathrooms'] ]
fifth_listing = normalized_listings.iloc[4][ ['accommodates', 'bathrooms'] ]

first_fifth_distance = distance.euclidean( first_listing, fifth_listing )
print(f'La distancia entre el primer elemento y el quinto de nuestro conjunto de datos es: {first_fifth_distance}')

La distancia entre el primer elemento y el quinto de nuestro conjunto de datos es: 1.4968643297650204


### Introduccion a Scikit-learn

Hasta ahora hemos construido el modelo de KNN desde cero. Pero para ser mas productivo e iterar mas rapido, podemos hacer uso <br>
de la biblioteca _sickit-learn_, la cual contiene funciones para todos los principales algoritmos de ML y un flujo de trabajo simple <br>
y unificado.<br>

El flujo de trabajo de scikit-learn consta de 4 pasos pricipales:
- Instalar el modelo de aprendizaje automatico especifico que se desea.
- Ajustar el modelo a los datos de entrenamiento.
- Utilizar el modelo para hacer predicciones.
- Evaluar la precision de las predicciones.

_**IMPORTANTE**_: <br>
Cualquier modelo que nos ayude apredecir valores numericos, como el precio de venta en nuestro caso, se conoce como **modelo de regresion**. <br>
La otra clase principal de modelos de ML supervisado se denomina de **clasificacion**, en la que intentamos predecir una etiqueta a partir <br>
de un conjunto fijo de etiquetas (por ejemplo, el tipo de sangre o el sexo).

In [31]:
from sklearn.neighbors import KNeighborsRegressor
# Algunos de los hiperparametros que le podemos configurar son: algorithm, n_neighbors, p, entre otros.
knn = KNeighborsRegressor(algorithm='brute') # Regressor hace referencia a que es un modelo de la clase regresion.

#### Ajuste de Modelo y Realizacion de predicciones

Ahora podemos ajustar (fit_method) el modelo. Para ello se utiliza el metodo **fit(X,y)** donde: 
- X: Es un objeto tipo matriz, que contiene las columnas de caracteristicas que queremos utilizar del conjunto de entrenamiento.
- y: Objeto tipo lista, que contiene los valores objetivo correctos.

In [32]:
# Dividiendo los datos normalizados en datos de entrenamiento y prueba. 
train_df = normalized_listings.iloc[0:2792]
test_df = normalized_listings.iloc[2792:]

# Seleccionando las columnas de caracteristicas que se desean usar para 'ajustar' (entrenar) el modelo.
train_features = train_df[[ 'accommodates', 'bathrooms' ]]
train_target = train_df['price']

# Ajustando modelo
knn.fit(train_features, train_target)

KNeighborsRegressor(algorithm='brute')

**NOTA**: Los datos que se le pasen al metodo fit, no deben contener valores nulos.

#### Haciendo predicciones

Ahora podemos usar el metodo _**predict()**_ para hacer las predicciones, este metodo solo tiene un parametro requerido:
- Objeto tipo matriz, que contiene las columnas caracteristicas del conjunto de datos sobre el que queremos hacer predicciones.

**IMPORTANTE**: El numero de columnas de carateristicas que se utiliza durante el entrenamiento y prueba debe coincidir o scikit-learn devolvera un error.

In [33]:
predictions = knn.predict(test_df[[ 'accommodates', 'bathrooms' ]]) #Devuelve un array NumPy
print(predictions[0:10])

[139.  194.  194.  159.6  67.8 159.6 139.  174.8 198.6 194. ]


#### Aplicando todo lo aprendido de Scikit-learn

In [34]:
from sklearn.neighbors import KNeighborsRegressor

train_df = normalized_listings.iloc[0:2792]
test_df = normalized_listings.iloc[2792:]
train_columns = ['accommodates', 'bathrooms']

# Instanciando modelo de ML
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

# Ajustando modelo a los datos.
knn.fit(train_df[train_columns], train_df['price'])

# Haciendo predicciones
predictions = knn.predict(test_df[train_columns])
print(predictions[:20])

[139.  194.  194.  159.6  67.8 159.6 139.  174.8 198.6 194.  194.   67.8
 194.  159.6 159.6 194.   67.8  67.8 146.2 450. ]


#### Calculo del MSE con Scikit-learn

La funcion _**sklearn.metrics.mean_squared_error()**_ toma dos entradas:
- Objeto tipo lista, que representa los valores verdaderos.
- Objeto tipo lista, que representa los valores predichos usando el modelo.

In [35]:
from sklearn.metrics import mean_squared_error

two_features_mse = mean_squared_error( test_df['price'], predictions )
two_features_rmse = two_features_mse ** (1/2)
print(f'El MSE para el KNN con Scikit-learn es: {two_features_mse} y el RMSE es: {two_features_rmse}')

El MSE para el KNN con Scikit-learn es: 14844.169783845278 y el RMSE es: 121.8366520544835


Si recordamos los modelos anteriores, en los cuales solo se usaron una caracteristica (**caso Univariante**), podemos concluir que <br>
este modelo acabo funcionando mejor (menor puntuacion de error). <br>

#### Modelo con mas caracteristicas.

Ahora vamos a entrenar un modelo con _4_ caracteristicas:
- accommodates.
- bedrooms.
- bathrooms.
- number_of_reviews.

In [36]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

train_df = normalized_listings.iloc[0:2792]
test_df = normalized_listings.iloc[2792:]
train_columns = [ 'accommodates', 'bathrooms', 'bedrooms', 'number_of_reviews' ]

# Instanciando el modelo KNN de ML.
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

# Ajustando modelo a los datos.
knn.fit(train_df[train_columns], train_df['price'])

# Haciendo predicciones.
predictions = knn.predict(test_df[train_columns])

# Evaluando modelo.
mse = mean_squared_error(test_df['price'], predictions)
rmse = mse ** (1/2)
print(f'El modelo con 4 caracteristicas:\n{train_columns} \nTiene un MSE de: {mse} y un RMSE de {rmse}')

El modelo con 4 caracteristicas:
['accommodates', 'bathrooms', 'bedrooms', 'number_of_reviews'] 
Tiene un MSE de: 11196.015335608647 y un RMSE de 105.81122499814775


#### Utilizacion de todas las caracteristicas
Hasta ahora existe la tendencia que al aumentar las caracteristicas, los valores de MSE y RMSE disminuyen. <br>
Llevemos esto al extremo y utilizemos todas las caracteriticas.

In [37]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

train_df = normalized_listings.iloc[0:2792]
test_df = normalized_listings.iloc[2792:]

# Abstrayendo todas las columnas independientes.
features = train_df.columns.tolist()
features.remove('price')

# Instanciando modelo.
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

# Ajustando modelo.
knn.fit(train_df[features], train_df['price'])

# Haciendo predicciones.
all_features_predictions =  knn.predict(test_df[features])

# Evaluando modelo.
mse = mean_squared_error(test_df['price'], all_features_predictions)
rmse = mse ** (1/2)
print(f'El modelo con todas las caracteriticas, arrojo un MSE de {mse} y un RMSE de {rmse}')


El modelo con todas las caracteriticas, arrojo un MSE de 10940.285961319682 y un RMSE de 104.59582191139224


**Curiosamente, el MSE y el RMSE, sigui disminuyendo, aunque esto no es siempre asi, el usar mas caracteristicas no siempre mejorara las <br>
precciones de nuestro modelo.**

Asi que deberiamos considerar lo siguiente:
- Seleccionar los atributos o caracteriticas relevantes para entrenar el modelo.

Hasta ahora hemos visto que para mejorar las predicciones del modelo se usaron distintas y mas o menos caracteristicas. Lo que se conoce como _**seleccion de caracteristicas**_. Ademas de esto, podemos ajustar otras otros parametros, los llamados _**hiperparametros.**_

### Exportando DataFrames a CSV.

In [38]:
train_df.to_csv('./train_df.csv')
test_df.to_csv('./test_df.csv')